<a href="https://colab.research.google.com/github/rishab8218/--VocalInk-/blob/main/Mavericks_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U openai-whisper
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 15.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=35bafa2b9b307d8cbf081d6803d2944eaaa1781072fd8d588b2b600371a9cd33
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [ ]:
# Install whisper and pydub (if not already installed)
!pip install -q whisper pydub

# Ensure ffmpeg is installed, which is essential for pydub and whisper
!sudo apt-get update -q && sudo apt-get install -y ffmpeg -q

print("Required libraries and ffmpeg installed.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,233 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadc

In [ ]:
from IPython.display import HTML, Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import write as write_wav
import io
import datetime

# --- JavaScript for Audio Recording ---
# This JavaScript code creates a recording interface and sends the audio data back to Python.
# It records in WAV format.
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = () => resolve(reader.result)
  reader.readAsDataURL(blob)
})
var record = async (time) => {
  let stream       = await navigator.mediaDevices.getUserMedia({ audio: true })
  let recorder     = new MediaRecorder(stream)
  let div          = document.createElement('div')
  let start        = document.createElement('button')
  let stop         = document.createElement('button')
  let recordedChunks = []
  start.textContent = 'Start Recording'
  stop.textContent  = 'Stop Recording'
  div.appendChild(start)
  div.appendChild(stop)
  document.body.appendChild(div)

  return new Promise(async resolve => {
    start.onclick = () => {
      recorder.start()
      start.textContent = 'Recording...'
      start.disabled = true;
      stop.disabled = false;
    }
    stop.onclick =  () => {
      recorder.stop()
      start.textContent = 'Start Recording'
      start.disabled = false;
      stop.disabled = true;
    }
    recorder.ondataavailable = e => recordedChunks.push(e.data)
    recorder.onstop = async () => {
      let blob = new Blob(recordedChunks, { type: 'audio/wav' })
      let url  = await b2text(blob)
      resolve(url)
    }
  });
}
"""

def record_audio_colab(filename="recorded_audio.wav"):
    """
    Displays a record/stop button, records audio from browser, and saves it as WAV.
    Returns the filename of the recorded audio.
    """
    print("Press 'Start Recording' to begin, 'Stop Recording' to finish.")
    display(Javascript(RECORD))
    audio_url = eval_js('record(1)') # The '1' is a dummy parameter for JS, actual recording duration is user controlled
    data = b64decode(audio_url.split(',')[1])

    # Generate a unique filename using timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"recorded_audio_{timestamp}.wav"

    with open(output_filename, 'wb') as f:
        f.write(data)
    print(f"Audio saved as '{output_filename}'")
    return output_filename

# Optional: Add a CSS style for better button appearance
display(HTML("""
<style>
button {
  background-color: #4CAF50; /* Green */
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
  border-radius: 8px;
}
button:hover {
  background-color: #45a049;
}
button:disabled {
  background-color: #cccccc;
  cursor: not-allowed;
}
</style>
"""))

In [ ]:
import whisper
import os
print(os.getcwd())
!ls -l
def transcribe_recorded_audio():
    audio_file = record_audio_colab()
    if not os.path.exists(audio_file):
        print("No audio file was recorded or found.")
        return

    # Load the Whisper model
    print("\nLoading Whisper model...")
    # Consider using a smaller model like "tiny" or "base" for faster processing
    # "medium" or "large" offer better accuracy but take longer to load and run
    model = whisper.load_model("base")
    print("Whisper model loaded.")

    # Transcribe the recorded audio
    print(f"\nTranscribing '{audio_file}' with Whisper...")
    try:
        result = model.transcribe(audio_file)
        print("\n--- Transcription Result ---")
        print(result["text"])
    except Exception as e:
        print(f"\nAn error occurred during transcription: {e}")
        print("Please ensure your audio is clear and not too short.")

    # Optional: Clean up the recorded audio file after transcription
    # os.remove(audio_file)
    # print(f"Removed temporary audio file: {audio_file}")



/content
total 4
drwxr-xr-x 1 root root 4096 Dec  9 14:42 sample_data


In [ ]:
# --- Run the entire process ---
print("Ready to record and transcribe!")
transcribe_recorded_audio()

Ready to record and transcribe!
Press 'Start Recording' to begin, 'Stop Recording' to finish.


<IPython.core.display.Javascript object>

Audio saved as 'recorded_audio_20260109_134905.wav'

Loading Whisper model...


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 102MiB/s]


Whisper model loaded.

Transcribing 'recorded_audio_20260109_134905.wav' with Whisper...

--- Transcription Result ---
 agents in isolation, one using landchained, another using Creo AI, another build on OpenAIA. So if they all spoke different languages, none of them could collaborate. It would be like trying to run the internet without a standard like HTTP, complete chaos. That's why, standardized.
